<a href="https://colab.research.google.com/github/ElMartinez31/Data_Science/blob/main/Move_Pytorch_To_Production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mettre en production un modèle entraîné

Transformer un prototype (Jupyter Notebook, script de recherche) en une application scalable utilisable par des utilisateurs ou des systèmes.

Exemple : Un modèle RL entraîné pour jouer à Mario pourrait être déployé dans un jeu autonome ou un robot.

Optimiser les performances

Réduire la latence et la consommation mémoire (via TorchScript, ONNX, ou la quantification).

Exemple : Un modèle PyTorch converti en TorchScript pour fonctionner sur mobile.

Compatibilité avec différents environnements

Faire fonctionner le modèle sur des serveurs, edge devices (téléphones, Raspberry Pi), ou le cloud (AWS, GCP).

Intégration avec des pipelines industrielles

Connecter le modèle à des APIs (FastAPI, Flask), des bases de données, ou des outils comme Docker/Kubernetes.



🚀ONNX:

Interopérabilité : Exporter un modèle PyTorch vers ONNX permet de l'utiliser avec :

TensorFlow, MXNet, Scikit-learn (via onnxruntime)

Accélérateurs matériels (NVIDIA TensorRT, Intel OpenVINO)

Mobiles (Android/iOS via ONNX Runtime)

Optimisation : ONNX permet des optimisations (fusion d'opérations, quantification) pour des inférences plus rapides.

Portabilité : Le fichier .onnx est autonome et contient tout le graphe de calcul.

Exemple : Exporter un modèle PyTorch en ONNX

In [ ]:
import torch
import torch.nn as nn

# Définition d'un modèle simple
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Couche linéaire (10 entrées, 2 sorties)

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()
model.eval()  # Mode évaluation (important pour l'export ONNX)

In [ ]:
# Export en ONNX

# Exemple d'input (batch_size=1, input_dim=10)
dummy_input = torch.randn(1, 10)

# Export ONNX
torch.onnx.export(
    model,                     # Modèle PyTorch
    dummy_input,               # Input exemple
    "simple_model.onnx",       # Nom du fichier de sortie
    input_names=["input"],     # Nom de l'input
    output_names=["output"],   # Nom de l'output
    dynamic_axes={
        "input": {0: "batch_size"},  # Axe dynamique (taille de batch variable)
        "output": {0: "batch_size"},
    },
)


#torch.onnx.export: convertit le modèle en fichier .onnx.
#dummy_input: est un exemple de tensor pour tracer le modèle.
#dynamic_axes: permet de spécifier des dimensions variables (utile pour des batchs de tailles différentes).

Charger et utiliser un modèle ONNX

In [ ]:
import onnxruntime as ort

# Création d'un session ONNX Runtime
ort_session = ort.InferenceSession("simple_model.onnx")

# Input (doit correspondre au format attendu)
input_data = dummy_input.numpy()

# Inférence
outputs = ort_session.run(
    None,  # None car on veut toutes les sorties
    {"input": input_data},
)

print(outputs)

Vérifier l'export ONNX

In [ ]:
import onnx

model_onnx = onnx.load("simple_model.onnx")
onnx.checker.check_model(model_onnx)  # Vérifie que le modèle est valide
print(onnx.helper.printable_graph(model_onnx.graph))  # Affiche l'architecture

Cas pratique : Exporter un CNN (ex: ResNet)

In [ ]:
import torchvision

# Charger ResNet-18 pré-entraîné
model = torchvision.models.resnet18(pretrained=True)
model.eval()

# Exemple d'input (3 canaux, 224x224)
dummy_input = torch.randn(1, 3, 224, 224)

# Export ONNX
torch.onnx.export(
    model,
    dummy_input,
    "resnet18.onnx",
    input_names=["input"],
    output_names=["output"],
)

FastAPI et ONNX :

Un duo puissant pour le déploiement de modèles ML
En tant que data scientist, la combinaison de FastAPI et ONNX peut considérablement optimiser votre pipeline de déploiement de modèles. Voici comment ces deux technologies interagissent :

🔗 Relation entre FastAPI et ONNX
FastAPI et ONNX sont complémentaires mais servent des objectifs différents :

FastAPI : Framework pour créer des APIs web performantes (couche de service)

ONNX (Open Neural Network Exchange) : Format ouvert pour représenter des modèles ML (couche d'inférence)

🚀 Pourquoi utiliser ONNX avec FastAPI ?
Interopérabilité :

ONNX permet d'exporter des modèles depuis différents frameworks (PyTorch, TensorFlow, scikit-learn)

FastAPI expose ces modèles via une API standardisée

Performance optimisée :

Les modèles ONNX s'exécutent plus vite grâce à des optimisations spécifiques

FastAPI gère efficacement les requêtes entrantes

Déploiement multiplateforme :

ONNX fonctionne sur CPU/GPU et différents environnements

FastAPI fournit une interface REST indépendante de la plateforme

Exemple concret d'intégration

In [ ]:
from fastapi import FastAPI
import onnxruntime as ort
import numpy as np

app = FastAPI()

# Chargement du modèle ONNX
sess = ort.InferenceSession("modele.onnx")
input_name = sess.get_inputs()[0].name

@app.post("/predict")
async def predict(input_data: list):
    """Endpoint pour les prédictions ONNX"""
    # Conversion des données d'entrée
    input_array = np.array(input_data, dtype=np.float32)

    # Inférence ONNX
    outputs = sess.run(None, {input_name: input_array})

    return {"prediction": outputs[0].tolist()}

Workflow typique Data Science avec ONNX + FastAPI

Entraînement :

Développez votre modèle dans PyTorch/TensorFlow/scikit-learn

Conversion vers ONNX :

python
Copy
torch.onnx.export(model, dummy_input, "modele.onnx")
Déploiement avec FastAPI :

Créez une API autour du modèle ONNX

Bénéficiez des performances accrues d'ONNX Runtime

Consommation :

L'API peut être appelée par des applications web/mobiles

⚡ Avantages clés de cette combinaison
Latence réduite : Jusqu'à 10x plus rapide qu'un modèle Python natif

Compatibilité étendue : Fonctionne même avec des modèles entraînés sur d'autres plateformes

Économie de ressources : Moins de CPU/mémoire utilisés

Maintenance simplifiée : Un seul format de modèle à gérer

🛠️ Outils complémentaires utiles
ONNX Runtime : Moteur d'exécution optimisé pour les modèles ONNX

Hummingbird : Convertit les modèles sklearn en ONNX

Docker : Pour containeriser votre API FastAPI + modèle ONNX

Cette combinaison est particulièrement utile quand vous avez besoin :

De performances élevées en production

De déployer des modèles sur différentes plateformes

D'une solution standardisée pour servir différents types de modèles



Deployer un modele Pytorch to Fast API


In [ ]:
import torch
import torchvision

# 1. Charger ResNet-18 pré-entraîné
model = torchvision.models.resnet18(pretrained=True)
model.eval()  # Mode évaluation

# 2. Exemple de données (batch=1, RGB, 224x224)
dummy_input = torch.randn(1, 3, 224, 224)

# 3. Test du modèle avant export
output = model(dummy_input)
print("Classe prédite:", torch.argmax(output, dim=1).item())

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 111MB/s]


Classe prédite: 107


Théorie ONNX :

ONNX a besoin d'un exemple d'input pour :

Tracer le graphe de calcul

Déterminer les shapes des tensors intermédiaires

Valider que toutes les opérations sont supportées

In [ ]:
# Export ONNX avec batch dynamique
torch.onnx.export(
    model,
    dummy_input,
    "resnet18.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)


Ce qui se passe pendant l'export :

Tracing :

PyTorch exécute le modèle avec dummy_input

Enregistre toutes les opérations effectuées

Crée un graphe de calcul (DAG)

Validation :

Vérifie que toutes les opérations sont supportées par ONNX

Certaines opérations PyTorch complexes peuvent nécessiter des adaptations

Sérialisation :

Le graphe + poids sont sauvegardés au format protobuf (.onnx)

Points d'attention importants :

Compatibilité des opérations :

Certaines couches PyTorch n'ont pas d'équivalent direct ONNX

Solution : réimplémenter avec des opérations de base

Contrôle de flux :

Les boucles/conditions natives Python ne sont pas exportables

Utiliser torch.jit.script pour les modèles avec logique complexe

Shape Inference :

ONNX doit pouvoir déduire toutes les shapes intermédiaires

Problèmes fréquents avec les opérations de reshape dynamique

In [ ]:
import onnx

# Charger le modèle
onnx_model = onnx.load("resnet18.onnx")

# Valider le schéma
onnx.checker.check_model(onnx_model)

# Afficher le graphe
print(onnx.helper.printable_graph(onnx_model.graph))

ModuleNotFoundError: No module named 'onnx'

Créer une API avec FastAPI

In [ ]:
!pip install onnx
!pip install fastapi
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, File, UploadFile
import numpy as np
import onnxruntime as ort
from PIL import Image
import io
import urllib.request


app = FastAPI()


# Charger le modèle ONNX
sess = ort.InferenceSession("resnet18.onnx")

def preprocess_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes))
    img = img.resize((224, 224))  # Resize selon le modèle
    img = np.array(img).transpose(2, 0, 1)  # CHW format
    img = img.astype(np.float32) / 255.0  # Normaliser
    img = np.expand_dims(img, axis=0)  # Ajouter batch dimension
    return img

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    # Lire l'image uploadée
    image_bytes = await file.read()

    # Prétraiter l'image
    input_tensor = preprocess_image(image_bytes)

    # Faire la prédiction
    outputs = sess.run(
        None,
        {"input": input_tensor}
    )

    # Traiter les outputs (ex: obtenir la classe prédite)
    predicted_class = np.argmax(outputs[0])

    return {"predicted_class": int(predicted_class)}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

HTTPError: HTTP Error 404: Not Found